# DSCI 563 Lab 2: SGD (and optionally Newton's method)

### Assignment Topics
* Training a logistic regression model using numpy and pytorch
* Regularization
* Hyperparameter tuning
* Visualization of model weights
* Optionally: Newton's method for logistic regression

### Software Requirements
* Python (>=3.6)
* PyTorch (>=1.2.0)
* Jupyter (latest)
* Scikit Learn (>=0.23.2)
* Skorch (>=0.9)

### Submission Info
* TBD

## Getting started

Run the following code:

In [1]:
# Run this to import the required libraries 
import numpy as np
from sklearn.datasets import load_digits  
import matplotlib.pyplot as plt
import sklearn.metrics

import torch
import torch.nn as nn

# We'll use double values in our tensors
torch.set_default_dtype(torch.float64)

# Checks if GPU is available, otherwise use CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
torch.backends.cudnn.deterministic=True
print(device)

cpu


## Tidy Submission
rubric={mechanics:1}

To get the marks for tidy submission:

* Submit the assignment by filling in this jupyter notebook with your answers embedded
* Be sure to follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions)

## Before you get started...

In this assignment, you will be use numpy and vectorize many of the operations which we handled using for-loops in the lecture. For full score, many of the assignments require that you don't use loops. However, you will still get partial credit for solutions which use for-loops. 

If you can't come up with a vectorized solution after several attempts, it can be a good idea to implement a loop-based solution first and continue with the rest of the homework. If you have time, you can then come back to the tricky assignment later. 

## Assignment 1

In this final obligatory assignment, we'll build a logistic regression model for digit classification and implement training using stochastic gradient descent. As before, the digits we'll classify are 8-by-8 pixel grayscale images like the one below (representing the digit 0):

<img src="zero.png" alt="Zero" style="width: 100px;"/>

This time, we'll encode our examples into 65-dimensional feature vectors. The 64 first dimensions correspond to pixels in the input image and the last dimension corresponds to a bias feature which always has the activation 1. 

Run the following code to create the training, development and test set. We'll use 1000 examples for training, 400 for development and 397 examples for testing.

In [2]:
# Load 1789 digit images (X) with gold standard labels (y)
X, y = load_digits(return_X_y=True)

# Append a bias feature with activation 1 at then end of each example. 
# This gives us 65-dimensional feature vectors. 
X = np.append(X,np.ones((X.shape[0], 1)),axis=1)

# Form train, dev and test set
train_X = X[:1000]
dev_X = X[1000:1400]
test_X = X[1400:]

train_y  = y[:1000]
dev_y = y[1000:1400]
test_y = y[1400:]

for split, name in zip([train_X, dev_X, test_X], "train dev test".split()):
    print(f"{split.shape[0]} {split.shape[1]}-dim {name} examples")

1000 65-dim train examples
400 65-dim dev examples
397 65-dim test examples


### Assignment 1.1
rubric={accuracy:1, efficiency:1}

We'll start by defining a the softmax operation which converts a vector of activations into a probability distribution. 

The function `softmax(Y)` takes one argument: `Y` a numpy array of shape $m \times n$, representing $n$ activations for $m$ examples. It applies the softmax operation to each vector `Y[i]`:

$$ (r_1, ..., r_n) \mapsto (\frac{\exp(r_1)}{S}, ..., \frac{\exp(r_n)}{S}){\rm,\ where\ } S = \exp(r_1) + ... + \exp(r_n)$$

Note, to get full score for this assignment, you should present a fully vectorized solution (i.e. **no loops**).

Hint: [`numpy.repeat`](https://numpy.org/devdocs/reference/generated/numpy.repeat.html) and [`numpy.reshape`](https://numpy.org/devdocs/reference/generated/numpy.reshape.html) can be useful in the normalization step (i.e. when dividing by $S$).

In [30]:
x = np.array([[0,0,1],[1,0,0],[1,1,1]])
print('x', x)

print('x.sum(axis=1)', x.sum(axis=1))
print('x.sum(axis=1).reshape(3,1)', x.sum(axis=1).reshape(3,1))
print('x.sum(axis=1).reshape(-1,1)', x.sum(axis=1).reshape(-1,1))
S = x.sum(axis=1).reshape(-1,1)

print(x.shape[0]) # # of lines
print(x.shape[1]) # # of columns 
print(np.repeat(S, repeats=x.shape[1], axis=1))

x [[0 0 1]
 [1 0 0]
 [1 1 1]]
x.sum(axis=1) [1 1 3]
x.sum(axis=1).reshape(3,1) [[1]
 [1]
 [3]]
x.sum(axis=1).reshape(-1,1) [[1]
 [1]
 [3]]
3
3
[[1 1 1]
 [1 1 1]
 [3 3 3]]


In [52]:
# your code here
def softmax(Y):
    

A small test for your function.

In [62]:
# A test which your function should pass. Note, that simply passing the test does not 
# guarantee that your function is working fully correctly.
x = np.array([[0,0,1],[1,0,0],[1,1,1]])
sm_x = softmax(np.array([[0,0,1],[1,0,0],[1,1,1]]))
assert abs(sm_x[0].sum()) - 1 < 0.001
assert abs(sm_x[1].sum()) - 1 < 0.001
assert abs(sm_x[2].sum()) - 1 < 0.001

SUM of each row:
 [4.71828183 4.71828183 8.15484549] (3,)
SUM reshape:
 [[4.71828183]
 [4.71828183]
 [8.15484549]] (3, 1)
SUM reshape and repeat:
 [[4.71828183 4.71828183 4.71828183]
 [4.71828183 4.71828183 4.71828183]
 [8.15484549 8.15484549 8.15484549]] (3, 3)
Then, you can exp(vec) / SUM


### Assignment 1.2
rubric={accuracy:2, efficiency:1}

Next, we'll implement the negative log-likelihood loss function `nllloss`. The function takes two arguments:

* `y_hat`, an array of classifier output probabilities having dimension $m \times n$. There are $m$ examples and $n$ possible classes.
* `y` an array of $n$ gold standard labels, one for each example in `y_hat`:

You should return the sum of the negative log-likelihood of the gold standard labels:

$$ \mathcal{L} = -(\log(\hat{y}_{1}[y_1]) + ... + \log(\hat{y}_{m}[y_m])), $$

where $x[i]$ means the $i$-th element of vector $x$.

Note, to get full score for this assignment, you should present a fully vectorized solution (i.e. **no loops**).

Hint: [`numpy.arange`](https://numpy.org/doc/stable/reference/generated/numpy.arange.html) can be useful for picking the values `y_hat[0,y[0]]`, `y_hat[1,y[1]]`, ... `y_hat[m, y[m]]` using a single operation.

In [41]:
y_hat = np.zeros((3,3))
y_hat[0,1] = 1
y_hat[1,1] = 1
y_hat[2,2] = 1

print(y_hat)
print(y_hat[[0,1,2], np.array([1,1,2])]) # [0,1,2] -> line number which can be obtained by np.arange(# of lines); 
print(-np.log(y_hat[[0,1,2], np.array([1,1,2])]))
print(-np.log(y_hat[[0,1,2], np.array([1,1,2])]).sum() == 0.0)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[1. 1. 1.]
[-0. -0. -0.]
True


In [130]:
# your code here
def nllloss(y_hat, y):
    

A small test for your function

In [97]:
# A test which your function should pass. Note, that simply passing the test does not 
# guarantee that your function is working fully correctly.
y_hat = np.zeros((3,3))
y_hat[0,1] = 1
y_hat[1,1] = 1
y_hat[2,2] = 1
# print("Y_HAT:\n", y_hat)
# print("Gold position:\n", (np.array([1,1,2])))


# ar = np.arange(3)
# print("np.arange(3):\n", ar)

# Y_HAT[NP.ARANGE, GOLD_POSITION] = GOLD_VALUE

# print("Gold position's value in Y_HAT:\n", y_hat[ar, np.array([1,1,2])])
assert nllloss(y_hat,np.array([1,1,2])) == 0


Y_HAT:
 [[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Gold position:
 [1 1 2]
np.arange(3):
 [0 1 2]
Gold position's value in Y_HAT:
 [1. 1. 1.]
VALUE of the GOLD position: [1. 1. 1.]
Then, you can sum of all -np.log(values)


### Assignment 1.3
rubric={accuracy:1}

We'll now start constructing out logistic regression model. You'll first define an `init` function which initializes and returns an array of model weights. The function should take two arguments:

* `features`, the number of features (in our case, this will always be 65, that is, 64 pixels + the bias feature)
* `classes`, the number of output classes (in our case, this will always be 10 because there are 10 digits)

You should initialize all weights to 0 and return an array of shape `features x classes`.

In [45]:
features = 65
classes = 10
print(np.zeros((features, classes)))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0.

In [48]:
# your code here
def init(features, classes):
    

A small test for your function

In [10]:
# A test which your function should pass. Note, that simply passing the test does not 
# guarantee that your function is working fully correctly.
assert init(65,10).shape == (65, 10)
assert (init(65,10)**2).sum() == 0

We'll then initialize our model weights

In [49]:
W = init(65,10)

### Assignment 1.4
rubric={accuracy:1}

We'll then define a `forward` function for our logistic regression model. As arguments, the function will take:

* An array of $m$ examples `X` having shape $m \times n$, where $n$ is the number of features for an individual examples (in our case, 65).
* A weight matrix $W$ of shape $n \times c$, where $c$ is the number of output classes (10 in our case).

The function should use your `softmax` implementation and return an array of class probabilities having dimension `m x c`:

$$softmax(XW)$$

In [46]:
# your code here 
def forward(X,W):
    # use `softmax()` with X and W

A small test of your function

In [53]:
# A test which your function should pass. Note, that simply passing the test does not 
# guarantee that your function is working fully correctly.
W = init(65,10)
X = np.zeros((20,65))
assert forward(X,W).shape == (20, 10)
assert abs(forward(X,W)[0].sum() - 1) < 0.001

In [59]:
print(forward(X,W))
print('-')
print(forward(X,W)[0])
print('-')
print(forward(X,W)[0].sum())

[[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
-
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
-
1.0


In [54]:
forward(X,W)[0].sum() 

1.0

### Assignment 1.5
rubric={acuracy:3,efficiency:1}

Next, we'll implement the function `gradient` which returns the gradient of the negative log-likelihood loss with regard to the model parameters. The function takes four arguments:

* A single input example `x` having shape `1 x number_of_features` (1 x 65 in our case)
* A single label `y` (a regular integer like `0`).
* A weight matrix `W` of shape `number_of_features x number_of_classes` (65 x 10 in our case)
* The regularization strength `beta` for L2 regularization

You function should compute the gradient for each class and return all of them as an array having the same shape as `W`, that is, `number_of_features x number_of_classes`.

Recall that the gradient of the regularized loss for class $c$ and a single example $(x, y)$ is given by the following formula (for some reason, github can't render the matrices correctly but they should look okay in jupyter):

$$ \frac{\partial \mathcal{L}}{\partial w_c} = \begin{bmatrix} \frac{\partial \mathcal{L}(x)}{\partial w_{c,1}} \\ \vdots \\ \frac{\partial \mathcal{L}(x)}{\partial w_{c,n}}\end{bmatrix} = \begin{bmatrix} {(\hat{p}(c|x) - 1)\cdot x_i + 2\beta w_1} \\ \vdots \\ {(\hat{p}(c|x) - 1) \cdot x_{n} + 2 \beta w_n} \end{bmatrix}$$

when $c = y$, and

$$ \frac{\partial \mathcal{L}}{\partial w_c} = \begin{bmatrix} \frac{\partial \mathcal{L}(x)}{\partial w_{c,1}}\\ \vdots \\ \frac{\partial \mathcal{L}(x)}{\partial w_{c,n}}\end{bmatrix} = \begin{bmatrix} {\hat{p}(c|x)\cdot x_i + 2\beta w_1} \\ \vdots \\ {\hat{p}(c|x)\cdot x_{n} + 2\beta w_n} \end{bmatrix}$$

when $c \ne y$. 

You should use your `forward` function to compute the probabilities $\hat{p}(c|x)$ for each class $c$. 

Note, to get full score for this assignment, you should present a fully vectorized solution (i.e. **no loops**).

In [75]:
x = np.ones((1, 65))
y = 0
W = np.zeros((65,10))
beta = 0.001

print('forward', forward(x,W))
# print('shape of forward', forward(x,W).shape)


forward [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
ys [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [60]:
# your code here
def gradient(x, y, W, beta):
    # (foward x @ W ) X + 2 beta W

A small test of your function

In [121]:
# A test which your function should pass. Note, that simply passing the test does not 
# guarantee that your function is working fully correctly.
x = np.ones((1, 65))
y = 0
W = np.zeros((65,10))
assert gradient(x,y,W,0.001).shape == W.shape
# gradient1(x,y,W,0.001) == gradient1(x,y,W,0.001)

(65, 1) (1, 10)
(65, 10)


### Assignment 1.6
rubric={accuracy:3,quality:1}

You'll now implement a training function `train_sgd`. It takes three arguments:

* `alpha`, the learning rate
* `beta`, the regularization strength
* `epochs`, the number of training epochs
* `quiet`, a boolean which indicates if the function should print information during training

The function starts by initializing model parameters $W$ using your `init` function (we need a 65 x 10 dimensional weight matrix). In every epoch, the function will iterate through the training data (`train_X` and `train_y`). It will compute the gradient of the negative log-loikelihood loss using your `gradient` function. It will then update the model parameters $W$ according to the formula:

$$W := W - \alpha \cdot \nabla \mathcal{L}$$

After every epoch, you should use your `nllloss` function to compute the loss over the training set. Unless `quiet=True`, print the average loss for a training example after each epoch (simply divide the loss by the number of training examples). The average loss for a training example should decrease through training (although it can oscillate a bit) and you should definitely attain an average loss << 1 at the end of training.  

After every epoch, you should also evaluate your model on the development set. Tag the development set using your `forward` function and compute the accuracy using [`sklearn.metrics.accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html).

Keep track of the model parameters which produced the best development accuracy. Return these parameters along with the best development accuracy after training. This is called early stopping.

To test your function, train for 10 epochs using `alpha=0.01` and `beta=0.001`. Your dev accuracy should be better than 90%.

```
def train_sgd():
    W = init
    best_W = np.array(W)
    best_acc = 0
    for x, y from train  (through epochs...)
        gradient:       Note that your x is (65,) --> (1,65)
        W = W - alpha *grad
    nllloss
    prediction 
    get accuracy 
    update accuracy 
    return W, best_accuracy
```

In [136]:
# Please use these constants instead of the numbers 10 and 65 in your code
# because magic numbers make your code less readable and less maintainable 
# (https://stackoverflow.com/questions/47882/what-is-a-magic-number-and-why-is-it-bad)
CLASSES = 10
FEATURES = 65

# your code here

def train_sgd(alpha, beta, epochs, quiet):

    best_acc = 0
    ...

print(f"Best development accuracy: {best_acc}%")

Epoch 1 loss: 0.19405969359582229, dev accuracy: 93.75
Epoch 2 loss: 0.13026479364421456, dev accuracy: 95.0
Epoch 3 loss: 0.10401230648225789, dev accuracy: 95.75
Epoch 4 loss: 0.08932121773098119, dev accuracy: 95.5
Epoch 5 loss: 0.08071591244627968, dev accuracy: 95.5
Epoch 6 loss: 0.0748273174424501, dev accuracy: 95.25
Epoch 7 loss: 0.07012783117859879, dev accuracy: 94.75
Epoch 8 loss: 0.06605684780132011, dev accuracy: 94.25
Epoch 9 loss: 0.06229368675692227, dev accuracy: 94.25
Epoch 10 loss: 0.05860935614082263, dev accuracy: 94.0
Best development accuracy: 95.75%


### Assignment 1.7
rubric={accuracy:2}

Perform full grid search for hyperparameter optimization of the learning rate `alpha` and regularization strength `beta`. Save the best parameters (i.e. the ones delivering optimal development accuracy) that you find as `best_W_sgd` and print the best `alpha`, `beta` configuration and best development accuracy. Test reasonable values for `alpha` and `beta` like `[1.0, 0.1, 0.01, 0.001, 0.0001]`. Use maximum epoch count `50` when calling `train_sgd`.

The grid search can take a few minutes. You should be able to get accuracy > 94%.

Finally, check performance on the test set (`test_X` and `test_y`) using `best_W_sgd`.

In [137]:
MAX_EPOCHS = 50

# your code here
best_acc = 0
best_alpha = 0
best_beta = 0
best_W_sgd = None


## Assignment 2

We will then visualize the weights which our logistic regression model learns for each digit class (0, 1, 2, 3, 4, 5, 6, 7, 8, 9). 

### Assignment 2.1
rubric={accuracy:2}

We will visualize weights in `best_W_sgd` using the function [`plt.imshow`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html). Our model has 65 features and each class has, therefore, 65 weights. The first 64 of these correspond to pixels in the digit images, whereas the last feature is a specialized bias feature. We will visualize only the first 64 weights. 

In order to visualize each weight vector `best_W_sgd[i]`, extract the 64 first elements of the weight vector. Then rearrange these into an 8x8 pixel image using `reshape` and plot using `imshow`. Please also print the digit label.  

It is recommended to use a diverging colormap like `seismic` (i.e. call `imshow` using the argument `cmap="seismic"`), because this will very clearly illustrate high and low weights. Also, 8x8 images are extremely grainy which can make them hard to interpret. You should use the argument `interpolation="bicubic"` when calling `imshow` to produce a smoother [interpolated](https://matplotlib.org/stable/gallery/images_contours_and_fields/interpolation_methods.html) visualization of the weights.

In [78]:
# best_W_sgd[i]

# your code here

# for i in range(10):
#     ...
#     wi =  .. 
#     plt.imshow(wi,cmap="seismic",interpolation="bicubic")
#     plt.show()


### Assignment 2.2
rubric={reasoning:2}

Do you think the weights learned by our model are interpretable? What do you think the high and low weights correspond to? Do you think some of the integers have more easily interpretable weight vectors than others? Why do you think that might be? 

Note, we're looking for your thoughts here and we want you to examine the weights and think about them. There is no "correct" answer but a perfect score requires that you explain your thought process (please keep your answer to two paragraphs, at most).

## Assignment 3

You should now compare our custom made logistic regression implementation to pytorch logistic regression. In pytorch terms, a logistic regression model is a neural network which has one linear layer followed by a softmax layer. You can use `nn.Sequential` to implement your pytorch model or build it from scratch (check last week's assignment for pointers). 

Once again, you are given a training function:

In [19]:
def train_torch(model, train_X, train_y,epochs,quiet=False):
    loss_function = nn.NLLLoss()
    sgd = torch.optim.SGD(model.parameters(), lr=0.001)
    for epoch in range(epochs):
        tot_loss = 0
        for x, y in zip(train_X, train_y):
            model.zero_grad()
            sys_y = model(x).log()
            loss = loss_function(sys_y, torch.tensor([y]))
            loss.backward()
            sgd.step()
            tot_loss += loss
        if not quiet:
            print(f"Epoch {epoch+1} loss: {tot_loss/len(train_y)}")

### Assignment 3.1
rubric={accuracy:1}

Implement your pytorch logistic regression model below. This will be very similar to the models in laste week's assignment but simpler because there is only one linear layer and no non-linearity (apart from the final softmax layer). 

Note that your model should take vectors of dimensionality 65 (or `FEATURES`) as input and generate output vectors of dimension 10 (or `CLASSES`).

In [20]:
# your code here
torch_lr = nn.Sequential( ... )

### Assignment 3.2
rubric={accuracy:1}

You should then convert the training, development and test sets (`train_X`, `dev_X` and `test_X`) into lists of pytorch tensors. Check last week's assignment for an example. 

Save your torch data as `torch_train_X`, `torch_dev_X` and `torch_test_X`.

In [21]:
# your code here
torch_train_X = 
torch_dev_X = 
torch_test_X = 

/var/folders/n9/3r6hrp015t9d1n6m8l36t0yh0000gn/T/ipykernel_12195/1700092446.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:233.)
  torch_train_X = [torch.tensor([x]) for x in train_X]


You should then train your model for 10 epochs using the `train_torch` function.

In [22]:
train_torch(torch_lr, torch_train_X, train_y, 100)

Epoch 1 loss: 0.7421488991586361
Epoch 2 loss: 0.16148844391350953
Epoch 3 loss: 0.1114043386890857
Epoch 4 loss: 0.08933341683984906
Epoch 5 loss: 0.07417186683326941
Epoch 6 loss: 0.0626775238868657
Epoch 7 loss: 0.05370857570053636
Epoch 8 loss: 0.04668567787101115
Epoch 9 loss: 0.04120821706374175
Epoch 10 loss: 0.03690530687982026
Epoch 11 loss: 0.0335146796052019
Epoch 12 loss: 0.030802930396568357
Epoch 13 loss: 0.028576748157307954
Epoch 14 loss: 0.026705113748319746
Epoch 15 loss: 0.025100195347737646
Epoch 16 loss: 0.023701169735824167
Epoch 17 loss: 0.022464839096111223
Epoch 18 loss: 0.021359626192797278
Epoch 19 loss: 0.020361826749215155
Epoch 20 loss: 0.01945333247030215
Epoch 21 loss: 0.01862017499746036
Epoch 22 loss: 0.01785149960740934
Epoch 23 loss: 0.017138791180271944
Epoch 24 loss: 0.016475283987456537
Epoch 25 loss: 0.015855521359912646
Epoch 26 loss: 0.015275035136248553
Epoch 27 loss: 0.014730114313940801
Epoch 28 loss: 0.014217636043497057
Epoch 29 loss: 0.01

### Assignment 3.3
rubric={accuracy:1}

You should now classify the development examples `torch_dev_X` using your torch model. You can copy the `classify` function from last week's assignment and apply it to the development and test set. Use [`sklearn.metrics.accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) to figure out the accuracy. Make sure that performance is similar to our custom made logistic regression model.

In [23]:
# your code here
def classify(data, model):
    res = []
    # ...
    return res

sys_dev_y = classify(torch_dev_X, torch_lr)
print(f"Dev acc: {sklearn.metrics.accuracy_score(dev_y,sys_dev_y)}")

sys_test_y = classify(torch_test_X, torch_lr)
print(f"Test acc: {sklearn.metrics.accuracy_score(test_y,sys_test_y)}")


Dev acc: 0.9525
Test acc: 0.9017632241813602


## Assignment 4

In this optional assignment, we'll implement Newton's method for model training. This is a bit more challenging than the assignments above.

### Assignment 4.1 Optional
rubric={accuracy:2,efficiency:1}

We'll start by computing the 2nd order derivative of the loss function, that is, the Hessian. Your `hessian` function takes two arguments:

* An array of input examples `data_X` which has shape `number_of_examples x number_of_features`. In our case, when calling the function with train_X, the array will have dimension 1000 x 65.
* The current model weights `W` having shape `number_of_features x number_of_classes` (65 x 10 in our case).

Recall that the hessian of the negative log-likelihood loss for the logistic regression model given a single training example $x$ and the class label $y$ is:

$$H(x,y) = \begin{bmatrix} \frac{\partial \mathcal{L}}{\partial w_1^2} & \frac{\partial \mathcal{L}}{\partial w_1 \partial w_2} & \ldots & \frac{\partial \mathcal{L}}{\partial w_1 \partial w_n} \\
\frac{\partial \mathcal{L}}{\partial w_2 \partial w_1} & \frac{\partial \mathcal{L}}{\partial w_2^2} & \ldots & \frac{\partial \mathcal{L}}{\partial w_2 \partial w_n} \\
 \vdots & \vdots & \ddots  & \vdots\\
\frac{\partial \mathcal{L}}{\partial w_n \partial w_1} & \frac{\partial \mathcal{L}}{\partial w_n \partial w_2} & \ldots & \frac{\partial \mathcal{L}}{\partial w_n^2} \end{bmatrix} = 
\begin{bmatrix} p(y|x)(1 - p(y|x))x_1x_1 & p(y|x)(1 - p(y|x))x_1x_2 & \ldots & p(y|x)(1 - p(y|x))x_1x_n\\
p(y|x)(1 - p(y|x))x_2x_1 & p(y|x)(1 - p(y|x))x_2x_2 & \ldots & p(y|x)(1 - p(y|x))x_2x_n \\
 \vdots & \vdots & \ddots  & \vdots\\
p(y|x)(1 - p(y|x))x_nx_1 & p(y|x)(1 - p(y|x))x_nx_2 & \ldots & p(y|x)(1 - p(y|x))x_nx_n \\ \end{bmatrix}$$

For each class $y$, you should compute the Hessian over the entire training set as an average of the individual $H(x,y)$:

$$H(y) = \frac{\Sigma_{k=1}^m H(x_m, y)}{n}$$

You should then return an array having shape `number_of_classes x number_of_features x number_of_features` which encodes the Hessian for each class $y$.

Note, to get full score for this assignment, you should use as few loops as possible. It is very tricky to avoid looping over the the classes `y` so you are allowed to do this, but try to avoid looping over the data in `data_X` because this is slow.

In [24]:
# your code here
def hessian(data_X, W):
    

A small test for your function

In [25]:
W = init(FEATURES, CLASSES)
assert hessian(train_X, W).shape == (CLASSES, FEATURES, FEATURES)

### Assignment 4.2 Optional
rubric={accuracy:2, quality:1}

We'll now implement Newton's method. You'll write a function `train_newton` which takes three arguments:

* `beta`, the regularization strength
* `epochs`, the number of training epochs
* `quiet`, a boolean which indicates if the function should print information during training

Note that there is no learning rate because Newton's method does not require one.

The function starts by initializing model parameters $W$ using your `init` function (we need a 65 x 10 dimensional weight matrix). 

In every epoch, you will need to first compute the average gradient $\nabla \mathcal{L}$ over all the examples in `data_X`. You can use your existing `gradient` function to compute a gradient for each example and average them. This should give you an array of shape `number_of_features x number_of_classes` (65 x 10 in our case).

You will then compute the Hessian $H$ using your function `hessian`. 

The function will then update the model parameters $W_c$ for class $c$ according to the formula:

$$W_c := W_c - \alpha \cdot  (H_c + \beta I)^{-1} \nabla \mathcal{L}_c$$

Here, $\beta$ is regularization. You can invert the matrix $H + \beta$ using [`numpy.linalg.inv`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html). To update all parameters, you can loop over the digit classes (0, 1, ..., 9) and update the parameters separately for each class.

After every epoch, you should use your `nllloss` function to compute the loss over the training set. Unless `quiet=True`, print the average loss for a training example after each epoch (simply divide the loss by the number of training examples). The average loss for a training example should decrease through training and you should definitely attain an average loss << 1 at the end of training.  

After every epoch, you should also evaluate your model on the development set. Tag the development set using your `forward` function and compute the accuracy using [`sklearn.metrics.accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html).

Keep track of the model parameters which produced the best development accuracy. Return these parameters along with the best development accuracy after training. This is called early stopping.

To test your function, train for 10 epochs using `beta=0.01`. Your dev accuracy should be better than 90%.

In [26]:
# your code here

def train_newton(beta, epochs, quiet):
    



    
    return W, best_acc

W, best_acc = train_newton(beta=1, epochs=20, quiet=False)
print(f"Best development accuracy: {best_acc}%")

Epoch 1 loss: 0.20608650471513504, dev accuracy: 93.5
Epoch 2 loss: 0.11563878315200629, dev accuracy: 95.5
Epoch 3 loss: 0.08571011664244932, dev accuracy: 95.5
Epoch 4 loss: 0.0694566082751175, dev accuracy: 96.25
Epoch 5 loss: 0.058914394941683684, dev accuracy: 95.75
Epoch 6 loss: 0.05140121811770905, dev accuracy: 95.75
Epoch 7 loss: 0.04571505565927129, dev accuracy: 95.75
Epoch 8 loss: 0.04123333179885361, dev accuracy: 95.75
Epoch 9 loss: 0.03759499028576157, dev accuracy: 95.75
Epoch 10 loss: 0.034574595465139, dev accuracy: 95.75
Epoch 11 loss: 0.032022537299291234, dev accuracy: 95.75
Epoch 12 loss: 0.02983505537829985, dev accuracy: 95.75
Epoch 13 loss: 0.027937530229234956, dev accuracy: 95.75
Epoch 14 loss: 0.026274761441801863, dev accuracy: 95.75
Epoch 15 loss: 0.02480493257833273, dev accuracy: 95.75
Epoch 16 loss: 0.023495736428455875, dev accuracy: 95.75
Epoch 17 loss: 0.022321785116260463, dev accuracy: 95.75
Epoch 18 loss: 0.021262833655848034, dev accuracy: 95.75


### Assignment 4.3 Optional
rubric={accuracy:1}

Optimization of the regularization strength `beta`. Save the best parameters (i.e. the ones delivering optimal development accuracy) that you find as `best_W_newton` and print the best `beta` and best development accuracy. Test reasonable values for `beta` like `[1.0, 0.1, 0.01, 0.001, 0.0001]`. Use maximum epoch count `50` when calling `train_newton`.

The grid search can take a few minutes. You should be able to get accuracy > 94%.

Finally, check performance on the test set (`test_X` and `test_y`).

In [27]:
MAX_EPOCHS = 50

# your code here


beta=1.0
Found best dev acc so far: 96.25
beta=0.1
beta=0.01
beta=0.001
beta=0.0001

Best alpha=0.001, beta=1.0, acc=96.25

Test acc: 90.6801007556675


### Assignment 4.5 Optional
rubric={reasoning:1}

Do you see a difference in accuracy between SGD and Newton training? Do you think this is surprising?